In [1]:
 import numpy as np

class SlidingPuzzle():
    def __init__(self, num_blocos):
        self.num_blocos = num_blocos

    def _encontra_posicao(self, estado, elemento):
        for i in range(self.num_blocos):
            for j in range(self.num_blocos):
                if estado[i, j] == elemento:
                    return i, j
        return None, None

    def verifica_estados(self, atual, objetivo):
        flag = True
        for i in range(self.num_blocos):
            if flag == False:
                break

            for j in range(self.num_blocos):
                if atual[i, j] != objetivo[i, j]:
                    flag = False;
                    break;

        return flag

    def expande_estado(self, atual):
        novos_estados = []
        linha, coluna = self._encontra_posicao(atual, 0)

        # Cima
        if linha > 0:
            novo_estado = np.copy(atual)
            nova_linha = linha - 1

            bloco_alvo = novo_estado[nova_linha, coluna]
            novo_estado[nova_linha, coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        # Baixo
        if linha < self.num_blocos - 1:
            novo_estado = np.copy(atual)
            nova_linha = linha + 1

            bloco_alvo = novo_estado[nova_linha, coluna]
            novo_estado[nova_linha, coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)


        # Esquerda
        if coluna > 0:
            novo_estado = np.copy(atual)
            nova_coluna = coluna - 1

            bloco_alvo = novo_estado[linha, nova_coluna]
            novo_estado[linha, nova_coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        # Direita
        if coluna < self.num_blocos - 1:
            novo_estado = np.copy(atual)
            nova_coluna = coluna + 1

            bloco_alvo = novo_estado[linha, nova_coluna]
            novo_estado[linha, nova_coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        return novos_estados

    #Calcula a distância de Manhattan entre o ponto atual e o objetivo
    def distanciaManhattan(self, atual, objetivo):
        total_dist = 0

        for i in range(self.num_blocos):
            for j in range(self.num_blocos):
                elemento = atual[i, j]
                linha, coluna = self._encontra_posicao(objetivo, elemento)

                dist = abs(linha - i) + abs(coluna - j)
                total_dist += dist

        return total_dist


### Busca Gulosa

In [2]:
import heapq

class BuscaGulosa():
    
    #Construtor
    def __init__(self, problema):
        self.problema = problema

    #Verifica se o estado já foi verificado
    def _verifica_visitado(self, estado, estados_visitados):
        for i in estados_visitados:
            if self.problema.verifica_estados(i, estado):
                return True
        return False

    def busca(self, inicio, fim):
        p_fila = []
        
        # H, ID, elemento. ID criado para diferenciarmos os estados, pois podem haver estados com a mesma heurística (H) 
        heapq.heappush(p_fila, (0, 0, inicio))

        id_estado = 0
        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0

        while not len(p_fila) == 0:
            atual = heapq.heappop(p_fila)[2] #Pega o primeiro elementos (variável 'inicio' definida acima)
            estados_visitados.append(atual) #Incluo o estado atual no vetor estados_visitados, para controle de onde já fui

            if self.problema.verifica_estados(atual, fim):
                solucao_encontrada = True
                break
            else:
                cont_estados += 1
                print("Visitando #", cont_estados)
                novos_estados = self.problema.expande_estado(atual) #Expandindo o estado atual

                for i in novos_estados:
                    if not self._verifica_visitado(i, estados_visitados): #Verifica se o estado já foi visitado
                        id_estado += 1
                        prioridade = self.problema.distanciaManhattan(i, fim) #Calcula a distância de Manhattan

                        print(i, "\nh = ", prioridade)
                        heapq.heappush(p_fila, (prioridade, id_estado, i)) #Adiciona o estado na fila 'p_fila'

        return solucao_encontrada, estados_visitados, cont_estados


### Main

In [3]:
problema = SlidingPuzzle(3)

start = np.matrix([[2, 8, 3], [1, 6, 4], [7, 0, 5]])
print(start, "\n********\n")

target = np.matrix([[1, 2, 3], [8, 0, 4], [7, 6, 5]])
print(target, "\n********\n")

gulosa = BuscaGulosa(problema)
solucao, visitados, steps = gulosa.busca(start, target)

if solucao:
    print(f"Solução encontrada em {steps}")
else:
    print("Solução não encontrada")

[[2 8 3]
 [1 6 4]
 [7 0 5]] 
********

[[1 2 3]
 [8 0 4]
 [7 6 5]] 
********

Visitando # 1
[[2 8 3]
 [1 0 4]
 [7 6 5]] 
h =  4
[[2 8 3]
 [1 6 4]
 [0 7 5]] 
h =  8
[[2 8 3]
 [1 6 4]
 [7 5 0]] 
h =  8
Visitando # 2
[[2 0 3]
 [1 8 4]
 [7 6 5]] 
h =  4
[[2 8 3]
 [0 1 4]
 [7 6 5]] 
h =  6
[[2 8 3]
 [1 4 0]
 [7 6 5]] 
h =  6
Visitando # 3
[[0 2 3]
 [1 8 4]
 [7 6 5]] 
h =  4
[[2 3 0]
 [1 8 4]
 [7 6 5]] 
h =  6
Visitando # 4
[[1 2 3]
 [0 8 4]
 [7 6 5]] 
h =  2
Visitando # 5
[[1 2 3]
 [7 8 4]
 [0 6 5]] 
h =  4
[[1 2 3]
 [8 0 4]
 [7 6 5]] 
h =  0
Solução encontrada em 5
